In [1]:
from equilibrator_api import ComponentContribution, 
cc = ComponentContribution()
for i in cc.search_compound('l-malate').identifiers:
    if i.registry.namespace == 'kegg':
        print(i.accession)

In [2]:
cc = ComponentContribution()
C00026 = cc.get_compound("kegg:C00026")

Compound(id=22, inchi_key=KPGXRSRHYNQIFN-UHFFFAOYSA-L)

In [3]:
C00026 = cc.get_compound("kegg:C00026")

In [15]:
C00026.identifiers[0]

CompoundIdentifier(registry=Registry(namespace=bigg.metabolite), accession=akg)

In [19]:
for i in cc.search_compound('l-malate').identifiers:
    if i.registry.namespace == 'kegg':
        print(i.accession)

C00149
